In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
import sklearn
import tqdm
from tqdm import tqdm 
import nltk
import warnings
warnings.filterwarnings("ignore") 
import cv2
from sklearn.model_selection import train_test_split
import PIL
from PIL import Image
import time

import tensorflow as tf
import keras
from keras.layers import Input,Dense,Conv2D,concatenate,Dropout,LSTM
from keras import Model
from tensorflow.keras import activations
import warnings
warnings.filterwarnings("ignore")
import nltk.translate.bleu_score as bleu

In [15]:
import shutil

In [3]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

In [4]:
chexnet_weights="/kaggle/input/fyp2-dataset/brucechou1983_CheXNet_Keras_0.3.0_weights.h5"

In [5]:
from tensorflow.keras.applications import DenseNet121
image_shape= (224,224,3)
image_input= Input(shape=(224,224,3))
base=DenseNet121(include_top=False,input_tensor=image_input,input_shape=image_shape)
pred=Dense(14,"sigmoid")(base.output)
chexnet_model=Model(inputs=base.input,outputs=pred)
chexnet_model.load_weights(chexnet_weights)
chexnet_model.summary()

29084464/29084464 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                             

In [6]:
final_chexnet_model=Model(inputs=chexnet_model.inputs,outputs=chexnet_model.layers[-2].output,name="Chexnet_model")   

In [7]:
image_folder = "/kaggle/input/fyp2-dataset/NLMCXR_png"
df_path = "/kaggle/input/fyp2-dataset/NLMCXR_reports/NLMCXR_reports/ecgen-radiology"

In [8]:
train=pd.read_csv("/kaggle/input/chexRayImpression/impression_train_final.csv")
test=pd.read_csv("/kaggle/input/chexRayImpression/impression_test_final.csv")

In [9]:
print(train.shape) 
print(test.shape)

(4420, 6)
(560, 5)


In [10]:
def image_feature_extraction(image1,image2):
  
  image_1 = Image.open(image1)
  image_1= np.asarray(image_1.convert("RGB"))
   
  image_2=Image.open(image2)
  image_2 = np.asarray(image_2.convert("RGB"))

  image_1=image_1/255
  image_2=image_2/255

  image_1 = cv2.resize(image_1,(224,224))
  image_2 = cv2.resize(image_2,(224,224))
    
  image_1= np.expand_dims(image_1, axis=0)
  image_2= np.expand_dims(image_2, axis=0)
    
   
  image_1_out=final_chexnet_model(image_1)
  image_2_out=final_chexnet_model(image_2)
  conc=np.concatenate((image_1_out,image_2_out),axis=2)
  image_feature=tf.reshape(conc, (conc.shape[0], -1, conc.shape[-1]))
   
  return image_feature

In [11]:
root_path = "/kaggle/input/fyp2-dataset/NLMCXR_png/"

In [14]:
test_image_features = []
start_time = time.time() 
for i in range(len(test)):
    print(i)
    image1 = root_path + test.iloc[i]["image_1"]
    image2 = root_path + test.iloc[i]["image_2"]
    features = image_feature_extraction(image1, image2)
    test_image_features.append(features)

# Convert the list of features to a numpy array
test_image_features = np.array(test_image_features)
end_time = time.time()  # End the timer
execution_time = end_time - start_time
print("Execution time:", execution_time, "seconds")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [16]:
np.savez("/kaggle/working/test_image_features.npz", test_image_features)

In [19]:
output_file_path = "/kaggle/working/test_image_features.npz"
destination_dir = "/kaggle/output/"
shutil.copy(output_file_path, destination_dir)

'/kaggle/output/test_image_features.npz'

In [31]:
output_file_path = "/kaggle/working/train_image_features2.npz"
destination_dir = "/kaggle/output/"
shutil.copy(output_file_path, destination_dir)

'/kaggle/output/train_image_features2.npz'

In [33]:

output_file_path = "/kaggle/output/train_image_features2.npz"
destination_dir = "/kaggle/working/"

# Check if the file already exists in the destination directory
if os.path.exists(output_file_path):
    # Remove the existing file
    os.remove(output_file_path)

# Move the file to the destination directory
shutil.move(output_file_path, destination_dir)

Error: Destination path '/kaggle/working/train_image_features2.npz' already exists

In [34]:
# Remove the file if it already exists
output_file_path = "/kaggle/working/train_image_features2.npz"
if os.path.exists(output_file_path):
    os.remove(output_file_path)

'/kaggle/output/train_image_features2.npz'

In [32]:
display.FileLink("/kaggle/working/train_image_features2.npz")

/kaggle/working/train_image_features2.npz

In [23]:
train_image_features = []
start_time = time.time() 
for i in range(len(train)):
    print(i)
    image1 = root_path + train.iloc[i]["image_1"]
    image2 = root_path + train.iloc[i]["image_2"]
    features = image_feature_extraction(image1, image2)
    train_image_features.append(features)

# Convert the list of features to a numpy array
train_image_features = np.array(train_image_features)
end_time = time.time()  # End the timer
execution_time = end_time - start_time
print("Execution time:", execution_time, "seconds")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [36]:
np.savez("/kaggle/working/train_image_features3.npz", train_image_features)

In [39]:
import shutil
shutil.move('/kaggle/working/train_image_features3.npz', '/kaggle/working/train_image_features3.npz.zip')

'/kaggle/working/train_image_features3.npz.zip'

In [40]:
from IPython.display import FileLink
file_path = "/kaggle/working/train_image_features3.npz.zip"
FileLink(file_path)

/kaggle/working/train_image_features3.npz.zip

In [42]:
output_file_path = "/kaggle/working/train_image_features2.npz"
destination_dir = "/kaggle/output/"
shutil.copy(output_file_path, destination_dir)

'/kaggle/output/train_image_features2.npz'

In [28]:
shutil.move(output_file_path, destination_dir)

Error: Destination path '/kaggle/output/train_image_features2.npz' already exists

In [41]:
import os
os.makedirs('/kaggle/output/', exist_ok=True)

In [ ]:
train_image_features = []
start_time = time.time() 
for i in range(len(train)):
    image1 = root_path + train.iloc[i]["image_1"]
    image2 = root_path + train.iloc[i]["image_2"]
    features = image_feature_extraction(image1, image2)
    train_image_features.append(features)

# Convert the list of features to a numpy array
train_image_features = np.array(train_image_features)
end_time = time.time()  # End the timer
execution_time = end_time - start_time
print("Execution time:", execution_time, "seconds")

In [12]:
k=100
#print(test_features[k])
one=test.iloc[k]["image_1"] 
two=test.iloc[k]["image_2"]

print(image_feature_extraction(root_path+one,root_path+two))  

tf.Tensor(
[[[0.         0.00149696 0.         ... 0.         0.90839916 0.        ]
  [0.         0.00234055 0.00178525 ... 0.         0.92380166 0.        ]
  [0.         0.00216008 0.00257611 ... 0.         0.92862934 0.        ]
  ...
  [0.         0.00108648 0.00499063 ... 0.         0.908353   0.        ]
  [0.         0.00186259 0.00144973 ... 0.         0.9096142  0.        ]
  [0.         0.00103528 0.00147697 ... 0.         0.9094954  0.        ]]], shape=(1, 98, 1024), dtype=float32)
